# This note book evaluate the k for problems

In [1]:
from rbnbr.problems.max_cut import MaxCutProblem
from rbnbr.solver.quantum.qrr import QRRMaxCutSolver
import numpy as np


In [2]:
qrr = QRRMaxCutSolver()
def get_k(problem):
    
    graph = problem.graph
    Z = qrr.get_corr_analytic(graph)
    
    evals, evecs = np.linalg.eigh(Z)
    # print(np.sum(eigenvectors - eigenvectors.T))
    evecs = evecs.T
    sign_vecs = (1 + np.sign(evecs)) // 2 # 0 / 1 solution
    
    
    k = 0 
    candidates = []
    for i in range(sign_vecs.shape[0]):
        sol = sign_vecs[i].astype(int).tolist()
        cost = problem.evaluate_solution(sol)
        candidates.append((cost, sol, evecs[i, :], evals[i], i))
    # print(evals)
        
    candidates.sort(reverse=True, key=lambda x: x[0])
    
    result = {}
    
    result['best_solution_index'] = candidates[0][4]
    result['best_solution_cost'] = candidates[0][0]
    
    # descending order by the eigenvalue
    candidates.sort(reverse=True, key=lambda x: x[3])
    
    result['max_eigenvalue_index'] = candidates[0][4]
    result['max_eigenvalue_cost'] = candidates[0][0]
    
    result['min_eigenvalue_index'] = candidates[-1][4]
    result['min_eigenvalue_cost'] = candidates[-1][0]
    
    # descending order by the spectral radius
    candidates.sort(reverse=True, key=lambda x: np.abs(x[3]))
    
    result['max_spectral_radius_index'] = candidates[0][4]
    result['max_spectral_radius_cost'] = candidates[0][0]
    
    result['min_spectral_radius_index'] = candidates[-1][4]
    result['min_spectral_radius_cost'] = candidates[-1][0]
    
    return result
    
                    


In [3]:
test_problem = MaxCutProblem.generate_random_maxcut_problem(10, p=0.5)
print(get_k(test_problem))

{'best_solution_index': 0, 'best_solution_cost': 14.0, 'max_eigenvalue_index': 9, 'max_eigenvalue_cost': 12.0, 'min_eigenvalue_index': 0, 'min_eigenvalue_cost': 14.0, 'max_spectral_radius_index': 0, 'max_spectral_radius_cost': 14.0, 'min_spectral_radius_index': 5, 'min_spectral_radius_cost': 9.0}


In [4]:
import pandas as pd
from tqdm.notebook import tqdm


n_each = 10

results = []

total_iterations = len(range(10, 61, 10)) * n_each
with tqdm(total=total_iterations, desc="Total Progress") as pbar:
    for n in range(10, 61, n_each):
        for _ in range(n_each):
            test_problem = MaxCutProblem.generate_random_maxcut_problem(n, p=0.5)
            result = get_k(test_problem)
            result['n_nodes'] = test_problem.N
            results.append(result)
            pbar.update(1)

df = pd.DataFrame(results)



Total Progress:   0%|          | 0/60 [00:00<?, ?it/s]

In [5]:
df

,best_solution_index,best_solution_cost,max_eigenvalue_index,max_eigenvalue_cost,min_eigenvalue_index,min_eigenvalue_cost,max_spectral_radius_index,max_spectral_radius_cost,min_spectral_radius_index,min_spectral_radius_cost,n_nodes
0,1,12.0,9,9.0,0,10.0,0,10.0,4,11.0,10
1,2,18.0,9,17.0,0,16.0,0,16.0,4,14.0,10
2,9,17.0,9,17.0,0,10.0,0,10.0,3,12.0,10
3,0,16.0,9,14.0,0,16.0,0,16.0,4,16.0,10
4,1,17.0,9,13.0,0,16.0,0,16.0,4,17.0,10
5,9,22.0,9,22.0,0,0.0,0,0.0,3,15.0,10
6,2,17.0,9,13.0,0,12.0,0,12.0,3,16.0,10
7,6,12.0,9,10.0,0,7.0,9,10.0,5,10.0,10
8,9,17.0,9,17.0,0,0.0,0,0.0,3,12.0,10
9,0,12.0,9,6.0,0,12.0,9,6.0,5,9.0,10
